In [2]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('lr3/Shakespeare_data.csv')

In [4]:
df.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"


In [5]:
filtered_data = df[~df['PlayerLine'].str.contains("ACT|SCENE", na=False)].copy()
filtered_data['PlayerLine'] = filtered_data['PlayerLine'].str.lower()
filtered_data['PlayerLine'] = filtered_data['PlayerLine'].apply(lambda x: re.sub(r'[^\w\s]', '', x))


In [6]:
train_data, test_data = train_test_split(filtered_data, test_size=0.2, random_state=42)

In [7]:
from collections import defaultdict, Counter
import numpy as np

class NgramModel:
    def __init__(self, n=3, smoothing=1):
        """
        Ініціалізація N-грамної моделі.
        :param n: Розмір N-грами.
        :param smoothing: Значення для Лапласового згладжування.
        """
        self.n = n
        self.smoothing = smoothing
        self.ngram_counts = defaultdict(Counter)
        self.context_counts = Counter()

    def train(self, text):
        """
        Навчання N-грамної моделі на заданому тексті.
        :param text: Список токенізованих речень.
        """
        for sentence in text:
            tokens = ['<s>'] * (self.n - 1) + sentence + ['</s>']
            for i in range(len(tokens) - self.n + 1):
                ngram = tuple(tokens[i:i+self.n])
                context = ngram[:-1]
                token = ngram[-1]
                self.ngram_counts[context][token] += 1
                self.context_counts[context] += 1

    def ngram_probability(self, context, token):
        """
        Обчислення ймовірності N-грами з Лапласовим згладжуванням.
        :param context: Контекст N-грами.
        :param token: Токен N-грами.
        :return: Ймовірність токена в даному контексті.
        """
        count = self.ngram_counts[context][token]
        total_count = self.context_counts[context]
        vocab_size = len(self.ngram_counts)
        return (count + self.smoothing) / (total_count + self.smoothing * vocab_size)

    def generate_text(self, length=10, k=5):
        """
        Генерація тексту на основі моделі з використанням top-k стратегії.
        :param length: Довжина тексту, який треба згенерувати.
        :param k: Кількість варіантів для вибору наступного токена.
        :return: Згенерований текст.
        """
        context = ('<s>',) * (self.n - 1)
        result = list(context)
        
        for _ in range(length):
            candidates = self.ngram_counts[context]
            if not candidates:
                break
            # Отримуємо top-k варіанти
            top_k = Counter({token: self.ngram_probability(context, token) for token in candidates}).most_common(k)
            tokens, probabilities = zip(*top_k)
            next_token = np.random.choice(tokens, p=np.array(probabilities) / sum(probabilities))
            result.append(next_token)
            context = (*context[1:], next_token)
            if next_token == '</s>':
                break
        
        return ' '.join(result).replace('<s>', '').replace('</s>', '').strip()

# Ініціалізація триграмної моделі для прикладу
ngram_model = NgramModel(n=3)

# Демонстрація тренування моделі на невеликому прикладі
sample_text = [
    "to be or not to be".split(),
    "that is the question".split(),
    "whether tis nobler in the mind to suffer".split()
]

# Тренуємо модель на прикладі
ngram_model.train(sample_text)

# Приклад генерації тексту на основі навченої моделі
generated_text = ngram_model.generate_text(length=10, k=5)
generated_text


'whether tis nobler in the mind to suffer'

In [8]:
class NgramModelWithPerplexity(NgramModel):
    def perplexity(self, test_text):
        """
        Обчислення перплексії на тестовому тексті.
        :param test_text: Список токенізованих речень для тестування.
        :return: Значення перплексії.
        """
        log_prob_sum = 0
        word_count = 0
        
        for sentence in test_text:
            tokens = ['<s>'] * (self.n - 1) + sentence + ['</s>']
            for i in range(len(tokens) - self.n + 1):
                ngram = tuple(tokens[i:i+self.n-1])
                token = tokens[i+self.n-1]
                prob = self.ngram_probability(ngram, token)
                log_prob_sum += np.log(prob) if prob > 0 else float('-inf')
                word_count += 1

        # Перплексія: експонента від -суми логарифмів ймовірностей, поділена на кількість слів
        perplexity = np.exp(-log_prob_sum / word_count) if word_count > 0 else float('inf')
        return perplexity

# Підготовка тексту для тренування і тестування
train_text = [line.split() for line in train_data['PlayerLine'].dropna()]
test_text = [line.split() for line in test_data['PlayerLine'].dropna()]

# Функція для тренування моделей з різними значеннями n та обчислення перплексії
def train_and_evaluate_ngram_models(train_text, test_text, n_values=[3, 5, 10]):
    results = {}
    
    for n in n_values:
        model = NgramModelWithPerplexity(n=n)
        model.train(train_text)
        perplexity = model.perplexity(test_text)
        results[n] = perplexity
    
    return results

# Навчання моделей для n = 3, 5, 10 та обчислення перплексії
model_results = train_and_evaluate_ngram_models(train_text, test_text, n_values=[3, 5, 10])
model_results


{3: 77227.74040444875, 5: 223365.61041386478, 10: 228597.952037149}

In [9]:
class NgramModelWithTopK(NgramModelWithPerplexity):
    def generate_text_with_top_k(self, length=10, k=5):
        """
        Генерація тексту з використанням top-k стратегії.
        :param length: Довжина тексту, який треба згенерувати.
        :param k: Кількість топ-варіантів для вибору наступного слова.
        :return: Згенерований текст.
        """
        context = ('<s>',) * (self.n - 1)
        result = list(context)

        for _ in range(length):
            candidates = self.ngram_counts[context]
            if not candidates:
                break
            # Обчислюємо ймовірності для кожного можливого наступного слова в контексті
            top_k = Counter({token: self.ngram_probability(context, token) for token in candidates}).most_common(k)
            tokens, probabilities = zip(*top_k)

            # Нормалізуємо ймовірності і вибираємо наступний токен на основі розподілу ймовірностей
            probabilities = np.array(probabilities) / sum(probabilities)
            next_token = np.random.choice(tokens, p=probabilities)

            result.append(next_token)
            context = (*context[1:], next_token)

            if next_token == '</s>':
                break

        return ' '.join(result).replace('<s>', '').replace('</s>', '').strip()

# Демонстрація використання top-k стратегії для генерації тексту з триграмною моделлю
ngram_model_top_k = NgramModelWithTopK(n=3)
ngram_model_top_k.train(train_text)

# Генерація тексту з top-5 стратегією
generated_text_top_k = ngram_model_top_k.generate_text_with_top_k(length=20, k=5)
generated_text_top_k


'to be'

In [10]:
import gradio as gr
import numpy as np

# Функція для генерації тексту
def generate_text(n, strategy, k, start_text, max_length):
    model = NgramModelWithTopK(n=n)
    model.train(train_text)  # Навчання моделі на train_text
    
    # Токенізуємо початковий текст
    start_tokens = start_text.lower().split()
    length = max_length
    
    if strategy == 'top-k':
        generated_text = model.generate_text_with_top_k(length=length, k=k)
    else:
        generated_text = model.generate_text(length=length)  # Greedy
    
    # Обчислення ентропії
    tokens = generated_text.split()
    entropies = []
    for i in range(len(tokens) - n + 1):
        context = tuple(tokens[i:i+n-1])
        token = tokens[i+n-1]
        prob = model.ngram_probability(context, token)
        entropies.append(-np.log(prob) if prob > 0 else float('inf'))
    
    avg_entropy = np.mean(entropies) if entropies else float('inf')
    return generated_text, avg_entropy

# Налаштування Gradio інтерфейсу
with gr.Blocks() as demo:
    gr.Markdown("# Генерація тексту за допомогою N-грамної моделі")
    n = gr.Slider(3, 10, step=1, label="Виберіть N для моделі", value=3)
    strategy = gr.Radio(["greedy", "top-k"], label="Виберіть стратегію генерації")
    k = gr.Slider(1, 20, step=1, label="Виберіть значення k для top-k стратегії", value=5)
    start_text = gr.Textbox(label="Початковий текст")
    max_length = gr.Slider(5, 50, step=1, label="Максимальна довжина згенерованого тексту", value=20)
    
    # Кнопка генерації та виведення результатів
    generate_button = gr.Button("Згенерувати текст")
    output_text = gr.Textbox(label="Згенерований текст")
    entropy = gr.Number(label="Ентропія згенерованого тексту")
    
    # Зв'язуємо функцію генерації з інтерфейсом
    generate_button.click(generate_text, inputs=[n, strategy, k, start_text, max_length], outputs=[output_text, entropy])

# Запуск демо-додатку
demo.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


to be or

wherefore art thou

all the world's a stage

love looks not

In [16]:
import nltk
from nltk.lm import Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.util import ngrams
import random

In [17]:
train_text_tokens = [list(line) for line in train_text]
test_text_tokens = [list(line) for line in test_text]

In [18]:
def train_ngram_model(n, train_text_tokens):
    # Підготовка n-грам та тренувальних даних
    train_data, padded_vocab = padded_everygram_pipeline(n, train_text_tokens)
    # Використання згладжування Лапласа
    model = Laplace(n)
    model.fit(train_data, padded_vocab)
    return model

In [19]:
def generate_text_nltk(model, n, start_text, max_length=15, strategy="top-k", k=5):
    # Токенізація початкового тексту
    context = start_text.split()
    generated = context.copy()
    
    for _ in range(max_length):
        # Створення контексту для n-грамної моделі
        context_ngram = list(ngrams(context, n - 1))[-1] if len(context) >= n - 1 else context
        
        if strategy == "greedy":
            # Найімовірніший наступний токен
            next_word = model.generate(text_seed=context_ngram)
        elif strategy == "top-k":
            # Отримання топ-k токенів і їх ймовірностей
            candidates = [(word, model.score(word, context_ngram)) for word in model.vocab]
            candidates = sorted(candidates, key=lambda x: x[1], reverse=True)[:k]
            words, probabilities = zip(*candidates)
            probabilities = np.array(probabilities) / sum(probabilities)
            next_word = np.random.choice(words, p=probabilities)
        
        if next_word == '</s>':
            break
        generated.append(next_word)
        context.append(next_word)
        
    return ' '.join(generated)


In [20]:
# Приклад використання
n = 3  # наприклад, триграма
model = train_ngram_model(n, train_text_tokens)
generated_text = generate_text_nltk(model, n, start_text="to be or", max_length=15, strategy="top-k", k=5)
print("Generated Text:", generated_text)

Generated Text: to be or <s> i am a gentleman


In [21]:
import nltk
from nltk.lm import Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.util import ngrams
import numpy as np

# Підготовка тексту для навчання (токенізація)
def prepare_data(train_text_tokens, n):
    train_data, padded_vocab = padded_everygram_pipeline(n, train_text_tokens)
    return train_data, padded_vocab

# Функція для створення та навчання n-грамної моделі
def train_ngram_model(n, train_text_tokens):
    train_data, padded_vocab = prepare_data(train_text_tokens, n)
    model = Laplace(n)
    model.fit(train_data, padded_vocab)
    return model

# Функція для генерації тексту за top-k стратегією
def generate_text_with_top_k(model, n, start_text, max_length=15, k=5):
    # Початковий контекст
    context = start_text.split()
    generated = context.copy()
    
    for _ in range(max_length):
        # Формуємо контекст для n-грамної моделі
        context_ngram = tuple(context[-(n-1):]) if len(context) >= n - 1 else tuple(context)
        
        # Отримуємо топ-k токени з їх ймовірностями
        candidates = [(word, model.score(word, context_ngram)) for word in model.vocab]
        candidates = sorted(candidates, key=lambda x: x[1], reverse=True)[:k]
        
        # Якщо жодного кандидата не знайдено, завершуємо генерацію
        if not candidates:
            break
        
        words, probabilities = zip(*candidates)
        probabilities = np.array(probabilities) / sum(probabilities)
        
        # Випадковий вибір наступного токена з top-k
        next_word = np.random.choice(words, p=probabilities)
        
        if next_word == '</s>':  # Завершення при досягненні кінця речення
            break
        
        generated.append(next_word)
        context.append(next_word)
    
    return ' '.join(generated)

# Приклад використання
train_text_tokens = [list(line.split()) for line in ["to be or not to be", "that is the question"]]
n = 3  # Триграмна модель
model = train_ngram_model(n, train_text_tokens)

# Генеруємо текст із початковими словами "to be or" і top-5 стратегією
generated_text = generate_text_with_top_k(model, n, start_text="to be or", max_length=15, k=5)
print("Generated Text:", generated_text)


Generated Text: to be or to not not <s> be to not not or <s> be not be to be


In [23]:
import gradio as gr
import nltk
from nltk.lm import Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.util import ngrams
import numpy as np

# Функція для підготовки та навчання моделі
def train_ngram_model(n, train_text_tokens):
    train_data, padded_vocab = padded_everygram_pipeline(n, train_text_tokens)
    model = Laplace(n)
    model.fit(train_data, padded_vocab)
    return model

# Функція для генерації тексту за top-k та greedy стратегіями
def generate_text(model, n, start_text, max_length=15, strategy="top-k", k=5):
    context = start_text.split()
    generated = context.copy()
    entropies = []
    
    for _ in range(max_length):
        context_ngram = tuple(context[-(n-1):]) if len(context) >= n - 1 else tuple(context)
        
        if strategy == "greedy":
            next_word = model.generate(text_seed=context_ngram)
        elif strategy == "top-k":
            candidates = [(word, model.score(word, context_ngram)) for word in model.vocab]
            candidates = sorted(candidates, key=lambda x: x[1], reverse=True)[:k]
            if not candidates:
                break
            words, probabilities = zip(*candidates)
            probabilities = np.array(probabilities) / sum(probabilities)
            next_word = np.random.choice(words, p=probabilities)
        
        if next_word == '</s>':
            break
        generated.append(next_word)
        context.append(next_word)
        
        # Обчислення ентропії для кожного обраного токена
        prob = model.score(next_word, context_ngram)
        if prob > 0:
            entropies.append(-np.log(prob))
    
    avg_entropy = np.mean(entropies) if entropies else float('inf')
    return ' '.join(generated), avg_entropy

# Підготовка навчального тексту
train_text_tokens = [list(line.split()) for line in ["to be or not to be", "that is the question"]]

# Створення Gradio інтерфейсу
def gradio_app(start_text, n, max_length, strategy, k):
    model = train_ngram_model(n, train_text_tokens)
    generated_text, avg_entropy = generate_text(model, n, start_text, max_length, strategy, k)
    return generated_text, avg_entropy

# Налаштування інтерфейсу
with gr.Blocks() as demo:
    gr.Markdown("# Генерація тексту за допомогою N-грамної моделі")
    start_text = gr.Textbox(label="Початковий текст")
    n = gr.Slider(3, 10, step=1, label="Виберіть N для моделі", value=3)
    max_length = gr.Slider(5, 50, step=1, label="Максимальна довжина згенерованого тексту", value=20)
    strategy = gr.Radio(["greedy", "top-k"], label="Виберіть стратегію генерації")
    k = gr.Slider(1, 20, step=1, label="Виберіть значення k для top-k стратегії", value=5)
    
    generate_button = gr.Button("Згенерувати текст")
    output_text = gr.Textbox(label="Згенерований текст")
    entropy = gr.Number(label="Ентропія згенерованого тексту")
    
    generate_button.click(gradio_app, inputs=[start_text, n, max_length, strategy, k], outputs=[output_text, entropy])

# Запуск Gradio демо
demo.launch()


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
